In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import files # Chỉ chạy trên Colab

# Cấu hình hiển thị biểu đồ
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.figure(figsize=(10, 5))

# 1. Tải dữ liệu
print("Upload file data.csv của bạn lên đây...")
uploaded = files.upload() # Nó sẽ hiện nút chọn file
df = pd.read_csv('data.csv')

# 2. Tiền xử lý dữ liệu (Preprocessing)
print("--- Đang xử lý dữ liệu ---")

# a. Chọn các đặc trưng (Features) quan trọng để train
# Có thể thêm bớt tùy ý, nhưng cần nhớ để làm giao diện nhập tương ứng
features = [
    'age', 'gender', 'daily_screen_time_hours', 
    'sleep_duration_hours', 'social_media_hours', 
    'work_related_hours', 'gaming_hours'
]
target = 'stress_level'

X = df[features].copy()
y = df[target].copy()

# b. Gom nhóm Stress Level (0-10) thành 3 mức độ (Phân lớp)
# 0-3: Thấp (Low) -> Nhãn 0
# 4-6: Trung bình (Medium) -> Nhãn 1
# 7-10: Cao (High) -> Nhãn 2
def bin_stress(val):
    if val <= 3: return 0
    elif val <= 6: return 1
    else: return 2

y = y.apply(bin_stress)
target_names = ['Low', 'Medium', 'High']
print("Phân bố nhãn sau khi gom nhóm:", y.value_counts())

# c. Xử lý cột Giới tính (Text -> Số)
# Male -> 0, Female -> 1, Other -> 2
le_gender = LabelEncoder()
X['gender'] = le_gender.fit_transform(X['gender'])

# d. Chia tập Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# e. Chuẩn hóa dữ liệu (Scaling) - Rất cần thiết
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Huấn luyện Mô hình
print("\n--- Đang huấn luyện mô hình ---")

# Model 1: Decision Tree
dt_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42)
dt_model.fit(X_train_scaled, y_train)
dt_pred = dt_model.predict(X_test_scaled)
dt_acc = accuracy_score(y_test, dt_pred)

# Model 2: Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)
rf_acc = accuracy_score(y_test, rf_pred)

print("\n--- 🚀 BẮT ĐẦU HUẤN LUYỆN K-MEANS (GOM CỤM) ---")
# 1. Chọn số cụm (Clusters)
# Chúng ta chọn k=3 tương ứng với 3 nhóm người dùng điển hình
kmeans = KMeans(n_clusters=3, random_state=42)

# Lưu ý: K-Means BẮT BUỘC phải dùng dữ liệu đã chuẩn hóa (X_train_scaled)
kmeans.fit(X_train_scaled)

# 2. Phân tích ý nghĩa từng cụm (Cluster Profiling)
# Máy chỉ biết cụm 0, 1, 2. Ta cần xem đặc điểm để đặt tên cho nó.
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_) # Đưa về số liệu gốc để dễ đọc
df_clusters = pd.DataFrame(cluster_centers, columns=features)
df_clusters['Cluster_Label'] = [0, 1, 2] # Gán nhãn

print("\n📊 ĐẶC ĐIỂM TRUNG BÌNH CỦA 3 NHÓM (Dùng để đặt tên trên App):")
print(df_clusters)

# A. So sánh độ chính xác
print(f"\n📊 KẾT QUẢ ĐỘ CHÍNH XÁC:")
print(f"- Decision Tree: {dt_acc*100:.2f}%")
print(f"- Random Forest: {rf_acc*100:.2f}%")

plt.figure(figsize=(6, 4))
plt.bar(['Decision Tree', 'Random Forest'], [dt_acc, rf_acc], color=['#3498db', '#2ecc71'])
plt.ylim(0, 1.1)
plt.title("So sánh độ chính xác (Accuracy)")
plt.ylabel("Accuracy Score")
for i, v in enumerate([dt_acc, rf_acc]):
    plt.text(i, v + 0.02, f"{v*100:.1f}%", ha='center', fontweight='bold')
plt.show()

# B.Classification Report)
print("\n📝 BÁO CÁO CHI TIẾT (Decision Tree):")
print(classification_report(y_test, dt_pred, target_names=target_names))

# C. Ma trận nhầm lẫn (Confusion Matrix) - Decision Tree
cm = confusion_matrix(y_test, dt_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Decision Tree')
plt.xlabel('Dự đoán (Predicted)')
plt.ylabel('Thực tế (Actual)')
plt.show()

print("\n📝 BÁO CÁO CHI TIẾT (Random Forest):")
print(classification_report(y_test, rf_pred, target_names=target_names))

# C. Ma trận nhầm lẫn (Confusion Matrix) - Random Forest
cm = confusion_matrix(y_test, rf_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Random Forest')
plt.xlabel('Dự đoán (Predicted)')
plt.ylabel('Thực tế (Actual)')
plt.show()

# D. Biểu đồ tầm quan trọng của đặc trưng (Feature Importance)
# Để trả lời câu hỏi: Cái gì ảnh hưởng nhất đến Stress?
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
sns.barplot(x=importances[indices], y=[features[i] for i in indices], hue=[features[i] for i in indices], legend=False,  palette='viridis')
plt.title('Yếu tố nào ảnh hưởng mạnh nhất đến Stress?')
plt.xlabel('Mức độ quan trọng')
plt.show()

# E. (Tùy chọn) Vẽ cây quyết định (Decision Tree Visualization)
print("\n🌳 Cấu trúc Cây quyết định (Rút gọn):")
plt.figure(figsize=(15, 8))
plot_tree(dt_model, feature_names=features, class_names=target_names, filled=True, max_depth=2, fontsize=10)
plt.title("Mô phỏng quy luật ra quyết định (Decision Tree)")
plt.show()

# Vẽ biểu đồ so sánh giờ dùng màn hình giữa các nhóm
sns.barplot(data=df_clusters, x='Cluster_Label', y='daily_screen_time_hours', palette='viridis')
plt.title("So sánh thời gian dùng màn hình giữa 3 nhóm")
plt.xlabel("Tên nhóm (Cluster)")
plt.ylabel("Giờ dùng trung bình")
plt.show()

# 4. Đóng gói và Xuất file (Export)
print("\n--- Đang đóng gói file ---")

folder_name = 'models'
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
os.makedirs(folder_name)

# Lưu 2 model
joblib.dump(dt_model, f'{folder_name}/decision_tree.pkl')
joblib.dump(rf_model, f'{folder_name}/random_forest.pkl')

# Lưu model k-means
joblib.dump(kmeans, 'models/kmeans_model.pkl')

# Lưu bộ Scaler (Quan trọng để app chạy đúng)
joblib.dump(scaler, f'{folder_name}/scaler.pkl')

# Lưu danh sách tên cột (để App biết thứ tự)
joblib.dump(features, f'{folder_name}/features.pkl')

# Nén và tải xuống
shutil.make_archive('models', 'zip', folder_name)
files.download('models.zip')
print("✅ Đã xong! Hãy lấy file zip giải nén vào thư mục dự án VS Code.")